# Silver_Data_Transform

In [0]:
df=spark.read.format("delta")\
        .option("InferSchema",True)\
        .option("header",True)\
        .load("abfss://bronze@adfnetflixdl.dfs.core.windows.net/netflix_titles")

In [0]:
display(df)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df=df.fillna({"duration_minutes":0, "duration_seasons":1})

In [0]:
df.display()

In [0]:
df=df.withColumn("duration_minutes", col('duration_minutes').cast(IntegerType()))\
     .withColumn("duration_seasons", col('duration_seasons').cast(IntegerType()))

In [0]:
df.printSchema()

In [0]:
display(df)

In [0]:
df=df.withColumn("head_title", split(col('title'),':')[0])

In [0]:
display(df)

In [0]:
df=df.withColumn("rating",split(col('rating'),'-')[0])

In [0]:
df.display()

In [0]:
df=df.withColumn("type_flag", when(col("type") == "TV Show", 1)\
    .when(col("type") == "Movie", 2)\
    .otherwise(0))

In [0]:
df.display()

In [0]:
from pyspark.sql import Window
window_spec= Window.orderBy(col("duration_minutes").desc())

In [0]:
df=df.withColumn("duration_ranking", dense_rank().over(window_spec))
display(df)

In [0]:
df.createOrReplaceTempView("temp_view")

### To create this table or view in another notebook during the same session use createOrReplaceGlobalView

In [0]:
df=spark.sql('''
             SELECT * FROM temp_view
             ''')
display(df)

In [0]:
df_vis=df.groupBy("type").agg(count("*").alias("total_count"))

In [0]:
display(df_vis)

In [0]:
df.display()

In [0]:
df.write.format("delta")\
        .mode("overwrite")\
        .option("path","abfss://silver@adfnetflixdl.dfs.core.windows.net/netflix_titles")\
        .save()